In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import Layer, Embedding, Input, Dense, Dropout, GlobalAveragePooling1D, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

1. Ladataan datasetti ja esikäsitellään data:
- Ladataan elukuva-arvosteludata ja rajataan sanavarston koko.
- Kaikki arvostelut täydennetään samanpituisiksi, jotta syötteet olisivat yhtenäisiä.

In [4]:
max_features = 10000
max_len = 250

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_len, padding='post')

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


2. Token- ja positioembeddays kerros huolehtii sanojen upotuksista (embedding) ja paikkaindeksin lisäämisestä.

In [5]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, seq_len, vocab_size, emb_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=emb_dim)
        self.pos_emb = Embedding(input_dim=seq_len, output_dim=emb_dim)

    def call(self, x_input):
        seq_len = tf.shape(x_input)[-1]
        positions = tf.range(start=0, limit=seq_len, delta=1)
        positions = self.pos_emb(positions)
        x_input = self.token_emb(x_input)
        return x_input + positions

3. Rakennetaan malli: Mallissa käytetään MultiHeadAttention-kerrosta residuaalisella yhteydellä ja kerrosnormalisoinnilla.

In [10]:
embed_dim = 32  # upotusten dimensio
num_heads = 2  # attention-päiden määrä
key_dim = embed_dim // num_heads  # yhden pään kyselyn/avaimen dimensio

# input-kerros, jossa jokainen arvostelu on 250 sanan vektori.
inputs = Input(shape=(max_len,))

x = TokenAndPositionEmbedding(max_len, max_features, embed_dim)(inputs)

# Multi-head attention-kerros: tämä oppii tärkeimmät sanaparitit arvosteluista.
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)

# residuaalinen yhteys: kiertää attention-kerroksen ja yhdistää alkuperäisen syötteen sen tulosten kanssa.
residual_output = Add()([x, attention_output])

# kerrosnormalisointi: normalisoi aktivointiarvot ja tasoittaa oppimisprosessia.
normalized_output = LayerNormalization()(residual_output)

# pooling kerros: muuntaa jokaisen arvostelun vakiopituiseksi, mikä on keskiarvo aktivaatiosta.
x = GlobalAveragePooling1D()(normalized_output)

# dropout-kerros: pudottaa 50% neuroneista estääkseen ylikuormitusta.
x = Dropout(0.5)(x)

# tiheä luokittelukerros: laskee lopullisen ennusteen.
outputs = Dense(1, activation='sigmoid')(x)

# mallin määrittely: käytetään adam-optimointia ja binääristä ristiin-entropiaa.
residual_model = Model(inputs=inputs, outputs=outputs)
residual_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# mallin yhteenveto: tulostetaan mallin arkkitehtuuri.
residual_model.summary()

# mallin koulutus: käytetään 5 epookkia ja 32 kokoisia eriä.
residual_model.fit(x_train, y_train, epochs=5, batch_size=32)

# mallin arviointi testidatalla: tarkistetaan mallin tarkkuus testidatasta.
print(f'Test accuracy = {residual_model.evaluate(x_test, y_test)[1]:.4f}')

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 250)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, 250, 32)   │    328,000 │ input_layer_3[0]… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 250, 32)   │      4,224 │ token_and_positi… │
│ (MultiHeadAttentio… │                   │            │ token_and_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 250, 32)   │          0 │ token_and_positi… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 250, 32)   │         64 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         33 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 332,321 (1.27 MB)

 Trainable params: 332,321 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 33ms/step - accuracy: 0.6937 - loss: 0.5558
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9193 - loss: 0.2119
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - accuracy: 0.9498 - loss: 0.1467
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - accuracy: 0.9631 - loss: 0.1146
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.9722 - loss: 0.0929
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8584 - loss: 0.4627
Test accuracy = 0.8561
